## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import os.path
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching
import gr_utils # Contains useful geometry functions
import gr_plot

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Input parameters

In [2]:
trailname = 'gr16' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)
delta = 0.005 # Tolerance around bounding box per trail section [deg]
points_per_batch = 100 # Subdivide the trail into batches of this many points

## Loading GPX file

In [3]:
filename_gpx = 'data_input/' + trailname + '.gpx'
filename_csv = 'data_output/' + trailname + '.csv'
if not os.path.isfile(filename_csv): # The GPX file was not processed into a clean CSV file before
    if not os.path.isfile(filename_gpx): # The GPX file does not exist, throw error
        raise ValueError(f'The GPX file <{filename_gpx}> was not found! Please make sure it exists.')
    else: # The GPX file exists, so convert it into a clean CSV file
        print(f'Converting GPX file <{filename_gpx}> into cleaned CSV file <{filename_csv}>...')
        gr_utils.process_gpx(filename_gpx,filename_csv)
        print('Completed conversion.')
print(f'Loading trail points from <{filename_gpx}>...')
trail = pd.read_csv(filename_csv) # Now read the cleaned CSV file into a DataFrame (latitude, longitude, elevation)
print('Finished loading.')

Loading trail points from <data_input/gr16.gpx>...
Finished loading.


## Matching GPX track to OSM network

In [4]:
# Matching GPX track to OSM network (uses _osm_network_download under the hood)
n_trail = len(trail) # Number of GPX points in the trail
n_batch = int(np.ceil(trail.shape[0]/points_per_batch)) # Number of batches to be run
for b in range(n_batch): # Using batch counter b
    
    # Define the range of GPX points to process in the current batch
    n1 = b*points_per_batch # First point of this batch
    n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)
    trail_section = trail.loc[n1:n2] # Select that range of GPX points
    trail_coords  = gr_mapmatch.trail_to_coords(trail_section) # Convert the points into a list of [lat, lon] pairs
    
    # Check if this batch was processed before
    batch_out = f'cache/{trailname}_{n1}to{n2}.csv'
    print(f'Handling {b} of {n_batch-1} that covers GPX track points {n1} through {n2}...')
    if os.path.isfile(batch_out): # It already exists
        print('   This batch was processed before, skipping.')
    else: # It does not exist, so process it
        network, segment_list = gr_mapmatch.match_batch(trail_section, trail_coords, delta)
        gr_utils.write_batch(batch_out, segment_list)
        print('   Finished this batch.')
        print('')

Handling 0 of 60 that covers GPX track points 0 through 100...


/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/osmnx/graph.py:488: UserWarning: the graph-level street_count attribute will likely be inaccurate when you set clean_periphery=False
  warnings.warn(msg)


   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 50 of 50...
   Handling route_list pair 65 of 65...
   Writing outputs to file
   Finished this batch.

Handling 1 of 60 that covers GPX track points 100 through 200...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 25 of 25...
   Handling route_list pair 26 of 26...
   Writing outputs to file
   Finished this batch.

Handling 2 of 60 that covers GPX track points 200 through 300...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 8 of 8...
   Handling route_list pair 8 of 8...
   Writing outputs to file
   Finished this batch.

Handling 3 of 60 that covers GPX track points 300 through 400...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 18 of 18...
   Handling route_list pair 19 of 19...
   Writing outputs to file
   Finished this batch.

Handling 4 of

   Finished this batch.

Handling 28 of 60 that covers GPX track points 2800 through 2900...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 13 of 13...
   Handling route_list pair 13 of 13...
   Writing outputs to file
   Finished this batch.

Handling 29 of 60 that covers GPX track points 2900 through 3000...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 17 of 17...
   Handling route_list pair 23 of 23...
   Writing outputs to file
   Finished this batch.

Handling 30 of 60 that covers GPX track points 3000 through 3100...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 11 of 11...
   Handling route_list pair 11 of 11...
   Writing outputs to file
   Finished this batch.

Handling 31 of 60 that covers GPX track points 3100 through 3200...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 5 of 5...
 

   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 16 of 16...
   Handling route_list pair 21 of 21...
   Writing outputs to file
   Finished this batch.

Handling 56 of 60 that covers GPX track points 5600 through 5700...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 14 of 14...
   Handling route_list pair 15 of 15...
   Writing outputs to file
   Finished this batch.

Handling 57 of 60 that covers GPX track points 5700 through 5800...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 24 of 24...
   Handling route_list pair 27 of 27...
   Writing outputs to file
   Finished this batch.

Handling 58 of 60 that covers GPX track points 5800 through 5900...
   Processing street network...
   Handling GPX point 100 of 100...
   Handling node_list pair 14 of 14...
   Handling route_list pair 16 of 16...
   Writing outputs to file
   Finished this batch.



## Merging sections

In [7]:
filename_roads = 'cache/' + trailname + '_roads.csv'
if not os.path.isfile(filename_roads): # The merged file does not exist
    print('Merged section file was not found, merging and saving...')
    data_roads = gr_utils.merge_roads(trailname, trail, points_per_batch) # Merge the different sections
    gr_utils.write_roads(trailname, data_roads) # Write the merged sections
    print('Saved.')
else: # The merged file does exist
    print('Loading merged section file...')
    data_roads = gr_utils.read_roads(trailname) # Read the merged sections
    print('Loaded.')

Loading merged section file...
Loaded.


## Removing backtrack segments

In [8]:
# Make a plot of the original and matched track
trail_coords  = gr_mapmatch.trail_to_coords(trail)
filepath = gr_plot.compare_tracks(trail_coords, data_roads)
IFrame(filepath, width=1000, height=500)

In [7]:
# Calculate if there are any repeats

## Gathering place information

In [ ]:
# Matching GPX track to OSM places (uses _osm_place_download under the hood)

In [ ]:
# Figure out how to process landuse into a useful statistic

## Establishing paved, traffic, and development type

## Saving completed data frame